<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 法1

In [ ]:
import dash
from dash import dcc, html, dash_table
import pandas as pd

# 原始数据
data = {
    "id": [1, 1, 1, 2, 3, 3, 4],
    "name": ["aaa", "aaa", "aaa", "bbb", "eee", "eee", "ggg"],
    "base": ["lot", "lot", "lot", "site", "lot", "lot", "site"],
    "wafer_id": ["c_1", "c_2", "c_3", "d_1", "f_1", "f_2", "h_1"]
}

df = pd.DataFrame(data)

# 按 'id', 'name', 'base' 进行分组，并合并 wafer_id
df_grouped = df.groupby(["id", "name", "base"], as_index=False).agg({
    "wafer_id": lambda x: ", ".join(x)  # 逗号合并 wafer_id
})

# 生成超链接
def generate_link(row):
    wafer_ids = row["wafer_id"]
    link = f"https://127.0.0.1:8050/user=a&wafer={wafer_ids}"

    if row["base"] == "lot":
        display_text = "all"
    else:
        display_text = wafer_ids  # 原本的 wafer_id 文字

    return f"[{display_text}]({link})"  # markdown 格式超链接

df_grouped["wafer_id"] = df_grouped.apply(generate_link, axis=1)

# 初始化 Dash 应用
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3("Wafer Data Table (Hyperlink Version)"),
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": "ID", "id": "id"},
            {"name": "Name", "id": "name"},
            {"name": "Base", "id": "base"},
            {"name": "Wafer ID", "id": "wafer_id", "presentation": "markdown"}  # 允许 markdown 显示超链接
        ],
        data=df_grouped.to_dict('records'),
        page_size=5,  # 分页
        sort_action="native",  # 允许排序
        filter_action="native",  # 允许筛选
        style_table={'overflowX': 'auto'}
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


### 法 2

In [ ]:
import dash
from dash import dcc, html, dash_table
import pandas as pd

# 原始数据
data = {
    "id": [1, 1, 1, 2, 3, 3, 4],
    "name": ["aaa", "aaa", "aaa", "bbb", "eee", "eee", "ggg"],
    "base": ["lot", "lot", "lot", "site", "lot", "lot", "site"],
    "wafer_id": ["c_1", "c_2", "c_3", "d_1", "f_1", "f_2", "h_1"]
}

df = pd.DataFrame(data)

# 1️⃣ 对每个 `id`，将所有 `wafer_id` 串联
df["wafer_id"] = df.groupby("id")["wafer_id"].transform(lambda x: ", ".join(x))

# 2️⃣ `drop_duplicates("id")` 仅保留 `id` 的第一行
df = df.drop_duplicates("id")

# 3️⃣ 生成超链接
def generate_link(row):
    wafer_ids = row["wafer_id"]
    link = f"https://127.0.0.1:8050/user=a&wafer={wafer_ids}"

    if row["base"] == "lot":
        display_text = "all"
    else:
        display_text = wafer_ids  # 原本的 wafer_id 文字

    return f"[{display_text}]({link})"  # markdown 格式超链接

df["wafer_id"] = df.apply(generate_link, axis=1)

# 初始化 Dash 应用
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3("Wafer Data Table (Hyperlink Version)"),
    dash_table.DataTable(
        id='table',
        columns=[
            {"name": "ID", "id": "id"},
            {"name": "Name", "id": "name"},
            {"name": "Base", "id": "base"},
            {"name": "Wafer ID", "id": "wafer_id", "presentation": "markdown"}  # 允许 markdown 显示超链接
        ],
        data=df.to_dict('records'),
        page_size=5,  # 分页
        sort_action="native",  # 允许排序
        filter_action="native",  # 允许筛选
        style_table={'overflowX': 'auto'}
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)
